#Web Scrapping from Walkscore.com

##Import Packages

In [1]:
!pip install js2py
!pip install selenium
!pip install convertapi

     |████████████████████████████████| 1.0 MB 5.8 MB/s 
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=26000 sha256=4425de9d10d03f2581315db867b726b6d72e8958d827021f94f02d0bbb18afb7
  Stored in directory: /root/.cache/pip/wheels/2a/80/ac/dcd2bdbd03dd2b7b7e2bf3e5afbda6a1ab7935bbce314969da
Successfully built pyjsparser
     |████████████████████████████████| 968 kB 7.2 MB/s 
     |████████████████████████████████| 138 kB 46.0 MB/s 
     |████████████████████████████████| 359 kB 14.7 MB/s 
     |████████████████████████████████| 3.6 MB 44.2 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol

In [2]:
import warnings
warnings.filterwarnings("ignore")

from IPython.display import display, HTML, IFrame
import math
import ast
import numpy as np
import pandas as pd
from shapely.geometry import MultiPoint
import js2py
from scipy import stats
import matplotlib.pyplot as plt
import bs4, re
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.request import urlopen
import requests
import json
%matplotlib inline

!wget -cq https://raw.githubusercontent.com/kirthi-b/Urban-Mobility-Index/main/Script/jscode/googlepolydecoder.js

In [3]:
import convertapi

convertapi.api_secret = 'x1InHbE9V178bARF' 

In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


##Direct Call Using API

In [ ]:
#payload = {'format': 'json', 'lat': '47.60', 'lon': '-122.32', 'transit': '1', 'bike': '1', 'wsapikey': '43cb9e445cace17ef525e7bf01cb71e5'}
#r = requests.get('https://api.walkscore.com/score', params=payload)

In [ ]:
#r.json()

##Webscrapping with BeautifulSoup

In [21]:
# Create an URL object
url = 'https://www.walkscore.com/WI/Madison/'
# Create object page
page = requests.get(url)

In [22]:
soup = BeautifulSoup(page.text, 'lxml')

In [23]:
# Obtain information from tag <table>
table1 = soup.find('table', id='hoods-list-table')
table1

<table class="tablesorter" id="hoods-list-table"> <thead class="emherit"> <tr> <th><p class="hidden-phone">Rank</p><p class="visible-phone">#</p><span></span></th> <th><p>Name</p><span></span></th> <th><p>Walk Score</p><span></span></th> <th><p>Transit Score</p><span></span></th> <th><p>Bike Score</p><span></span></th> <th class="population hidden-phone"><p>Population</p><span></span></th> </tr> </thead> <tbody> <tr class="hoods-list-item" id="item-153607"> <td class="rank">1</td> <td class="name"><a href="/WI/Madison/Downtown">Downtown</a></td> <td class="walkscore">92</td> <td class="transitscore"> 55 </td> <td class="bikescore"> 91 </td> <td class="population hidden-phone">17,587</td> </tr> <tr class="hoods-list-item" id="item-144005"> <td class="rank">2</td> <td class="name"><a href="/WI/Madison/State-Langdon">State-Langdon</a></td> <td class="walkscore">91</td> <td class="transitscore"> 56 </td> <td class="bikescore"> 88 </td> <td class="population hidden-phone">6,433</td> </tr> <

In [24]:
# Obtain every title of columns with tag <th>
headers = []
for i in table1.find_all('th'):
 title = i.text
 headers.append(title)

In [25]:
mydata = pd.DataFrame(columns = headers)

In [26]:
# Create a for loop to fill mydata
for j in table1.find_all('tr')[1:]:
 row_data = j.find_all('td')
 row = [i.text for i in row_data]
 length = len(mydata)      
 mydata.loc[length] = row

In [27]:
#Get URL of each neighborhood
mydata['URL'] = url + mydata['Name'].str.replace(" ", "_")

#Empty Polygon Column
mydata["Polygon"] = ""
mydata["EnPath"] = ""
mydata.head(2)

,Rank#,Name,Walk Score,Transit Score,Bike Score,Population,URL,Polygon,EnPath
0,1,Downtown,92,55,91,"17,587",https://www.walkscore.com/WI/Madison/Downtown,,
1,2,State-Langdon,91,56,88,"6,433",https://www.walkscore.com/WI/Madison/State-Lan...,,


##Locate Google Maps Overlay JavaScript File & Extract Polygon

In [28]:
#function for getting polygon from the google map overlay javascript file (and also locating the .js file for that)

def getpoly(url):
  #url would be the url of the neighborhood's/city's page
  data = urlopen(url)
  source = data.read().decode('utf-8')

  #regex to search
  regex = r"https:\/\/pp\.walk\.sc\/_\/s\/_p\/hood\/[a-z0-9]+\.js"

  matches = re.finditer(regex, source, re.MULTILINE)

  for matchNum, match in enumerate(matches, start=1):  
    jslink = match.group()

  #jslink is the javascript page link for the google map overlay
  data = urlopen(jslink)
  jspage = data.read().decode('utf-8')

  regex = r"(?<=window\.activeEntity = reconstituteEntity)\(.*\)"

  matches = re.finditer(regex, jspage, re.MULTILINE)

  for matchNum, match in enumerate(matches, start=1):
   polycode = match.group()

  #get overlay dictionary
  dic = json.loads(polycode[1:-1])

  #get encoded path string from dictionary
  path = dic['path']

  return path

In [29]:
def polydecode(en):
  eval_res, tempfile = js2py.run_file("googlepolydecoder.js")
  return tempfile.decodePolyline(en).to_list()

In [30]:
i = 0

while i < mydata.shape[0]:
  a = str(getpoly(mydata['URL'][i]))
  mydata['EnPath'][i] = a
  mydata['Polygon'][i] = polydecode(a)
  i = i+1

In [31]:
mydata.tail(3)

,Rank#,Name,Walk Score,Transit Score,Bike Score,Population,URL,Polygon,EnPath
35,36,Cherokee Park,9,18,28,"1,466",https://www.walkscore.com/WI/Madison/Cherokee_...,"[{'latitude': 43.15496, 'longitude': -89.38567...",oukfGlcq`PsDqB_DsDeG_Ki@oAcCqOgFsJqImFgByL_D{_...
36,37,North Lake Mendota,7,22,30,925,https://www.walkscore.com/WI/Madison/North_Lak...,"[{'latitude': 43.14123, 'longitude': -89.4015}...",u_ifGjft`Pw@??g@gJAcE_OqEkFqA_FrXuZrCqErAnChYo...
37,38,Hawks Landing Golf Club,5,11,20,"1,725",https://www.walkscore.com/WI/Madison/Hawks_Lan...,"[{'latitude': 43.03811, 'longitude': -89.53766...",e{teGjynaP@s\hi@j@Rj~@nODEt^yBNMXPbZiKIJxcAyU]...


In [32]:
mydata.to_pickle('/content/gdrive/MyDrive/Columbia/2 Spring 22/ML for Urban Data/ML Final/Project Files/Urban-Mobility-Index/Data/Testing/Madison/Madison.pkl')

In [33]:
def mapPara(txt,typ):
  dictionary = {"{":'(', "}":')', " ":'', "[":'', "]":''}
  pol = txt
  pol = pol.replace("'latitude'",'lat')
  pol = pol.replace("'longitude'",'lng')

  transTable = txt.maketrans(dictionary)
  txt = txt.translate(transTable)
  txt = txt.replace("'latitude':",'')
  txt = txt.replace("'longitude':",'')

  lst = list(ast.literal_eval(txt))
  points = MultiPoint(lst)
  b = points.bounds
  #n = b[2] - 0.25*(b[2]-b[0])
  #s = b[0] + 0.25*(b[2]-b[0])
  #e = b[3] - 0.25*(b[3]-b[1])
  #w = b[1] + 0.25*(b[3]-b[1])to
  x = 0.5*(b[0]+b[2])
  y = 0.5*(b[1]+b[3])

  if typ == 'bounds':
    return '{north:' + str(b[2]) + ', south:' + str(b[0]) + ', west:' + str(b[1]) + ', east:' + str(b[3]) + ',}'
    #return '{north:' + str(n) + ', south:' + str(s) + ', west:' + str(w) + ', east:' + str(e) + ',}'
  elif typ == 'center':
    return '{lat:' + str(x) + ', lng:' + str(y)+ '}'
  elif typ == 'polygon':
    return pol
  else:
    print ("Invalid Argument: Secondary Argument Must Be 'bounds','polygon' or 'center'")

In [34]:
def genHTML(p):
  url = 'https://kirthi-b.github.io/ML_Final/map.html'

  req = requests.get(url, 'html.parser')
  html = req.text

  html = html.replace("{p:}",str(mapPara(p,'polygon')))
  html = html.replace("{b:}",str(mapPara(p,'bounds')))
  html = html.replace("{c:}",str(mapPara(p,'center')))

  return html

In [35]:
i = 0
path_half = '/content/gdrive/MyDrive/Columbia/2 Spring 22/ML for Urban Data/ML Final/Project Files/Urban-Mobility-Index/Data/Testing/Madison/Maps/'

while i < mydata.shape[0]:
  url = path_half + 'HTML/' + str(mydata['Name'][i]) + '.html'
  with open(url, 'w') as f:
    f.write(genHTML(str(mydata['Polygon'][i])))
  i = i+1

In [36]:
i = 0

while i < mydata.shape[0]:
  url = path_half + 'HTML/' + str(mydata['Name'][i]) + '.html'
  dir = path_half + 'IMG/'
  convertapi.convert('png', {
      'File': url,
      'ConversionDelay': '4'
  }, from_format = 'html').save_files(dir)
  i = i+1